In [1]:
import pandas as pd

# import the data
data = pd.read_csv('./data/boystown.csv', sep=' ');

In [2]:
# transform categorical variables
data['sex'] -= 1;
data['dadjob'] -= 2;
data['momjob'] -= 2;

def map_gpa(gpa):
	if(gpa >= 3):
		return 0;
	return 1;

data['gpa'] = [map_gpa(gpa) for gpa in data['gpa']];

In [3]:
import numpy as np

# normalize the data
def normalize(x):
	x = np.array(x);
	return ((x - min(x)) / (max(x)-min(x))).tolist();

for label, content in data.items():
	data[label] = normalize(content);

In [4]:
# shuffle samples
data = data.sample(frac=1).reset_index(drop=True);

In [5]:
from sklearn.model_selection import train_test_split

# split train test samples
X = data.drop('gpa', axis=1);
Y = data['gpa'].values;

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.25);

In [6]:
from sklearn.neighbors import KNeighborsClassifier

# train for a range of k
evalArray = [];
for k in range(2, 15):
	model = KNeighborsClassifier(n_neighbors=k);
	model.fit(X, Y);

	trainScore = model.score(Xtrain, Ytrain);
	testScore = model.score(Xtest, Ytest);
	evalArray.append((k, trainScore, testScore));

In [7]:
# print results

evalArray = np.array(evalArray, dtype=[('k', int), ('train', float), ('test', float)]);

evalArray.sort(axis=0, order='test');
#print(evalArray);

#print(evalArray.tolist());
evalArray = evalArray[::-1];

print('K\tTrain\tTest');
for k, trainScore, testScore in evalArray.tolist():
	print(k,
		'\t', "{:.2}".format(trainScore),
		'\t', "{:.2}".format(testScore));

K	Train	Test
2 	 0.77 	 0.76
10 	 0.73 	 0.72
7 	 0.76 	 0.7
3 	 0.81 	 0.7
12 	 0.69 	 0.68
9 	 0.71 	 0.68
6 	 0.72 	 0.68
11 	 0.69 	 0.66
8 	 0.73 	 0.66
4 	 0.73 	 0.66
14 	 0.65 	 0.64
13 	 0.69 	 0.64
5 	 0.74 	 0.6
